In [ ]:
from google.colab import drive
drive.mount("/gdrive")
%cd /gdrive

In [ ]:
cd /gdrive/My Drive/renderresults

In [ ]:
!ls

In [30]:
#@title Setup
#@markdown Please configure your setup

blender_version = 'blender2.91' #@param ["blender2.91", "blender2.81", "blender2.80"] {allow-input: false}
path_to_blend = 'blend_files/fire.blend'  #@param {type: "string"}
output_path = 'renderresults/output###.png'  #@param {type: "string"}
gpu_enabled = True #@param {type:"boolean"}
cpu_enabled = False #@param {type:"boolean"}
#@markdown ---

In [31]:
if blender_version == "blender2.91":
    download_path="https://download.blender.org/release/Blender2.91/blender-2.91.0-linux64.tar.xz"
elif blender_version == "blender2.81":
    download_path="https://ftp.halifax.rwth-aachen.de/blender/release/Blender2.81/blender-2.81-linux-glibc217-x86_64.tar.bz2"
elif blender_version == "blender2.82":
    download_path="https://ftp.halifax.rwth-aachen.de/blender/release/Blender2.82/blender-2.82-linux64.tar.xz"

In [ ]:
!mkdir $blender_version

if blender_version == "blender2.91":
    !wget -O '{blender_version}.tar.xz' -nc $download_path
    !tar xf '{blender_version}.tar.xz' -C ./$blender_version --strip-components=1
else:
    !wget -O '{blender_version}.tar.bz2' -nc $download_path
    !tar -xf '{blender_version}.tar.bz2' -C ./$blender_version --strip-components=1

In [ ]:
import os

os.environ["LD_PRELOAD"] = ""

!apt update
!apt remove libtcmalloc-minimal4
!apt install libtcmalloc-minimal4
os.environ["LD_PRELOAD"] = "/usr/lib/x86_64-linux-gnu/libtcmalloc_minimal.so.4.3.0"

!echo $LD_PRELOAD

In [ ]:

!apt install libboost-all-dev
!apt install libgl1-mesa-dev
!apt install libglu1-mesa libsm-dev

In [35]:
data = "import re\n"+\
    "import bpy\n"+\
    "scene = bpy.context.scene\n"+\
    "scene.cycles.device = 'GPU'\n"+\
    "prefs = bpy.context.preferences\n"+\
    "prefs.addons['cycles'].preferences.get_devices()\n"+\
    "cprefs = prefs.addons['cycles'].preferences\n"+\
    "print(cprefs)\n"+\
    "# Attempt to set GPU device types if available\n"+\
    "for compute_device_type in ('CUDA', 'OPENCL', 'NONE'):\n"+\
    "    try:\n"+\
    "        cprefs.compute_device_type = compute_device_type\n"+\
    "        print('Device found',compute_device_type)\n"+\
    "        break\n"+\
    "    except TypeError:\n"+\
    "        pass\n"+\
    "#for scene in bpy.data.scenes:\n"+\
    "#    scene.render.tile_x = 64\n"+\
    "#    scene.render.tile_y = 64\n"+\
    "# Enable all CPU and GPU devices\n"+\
    "for device in cprefs.devices:\n"+\
    "    if not re.match('intel', device.name, re.I):\n"+\
    "        print('Activating',device)\n"+\
    "        device.use = "+str(gpu_enabled)+"\n"+\
    "    else:\n"+\
    "        device.use = "+str(cpu_enabled)+"\n"
with open('setgpu.py', 'w') as f:
    f.write(data)

In [ ]:
!sudo ./$blender_version/blender -P './setgpu.py'  -b '/gdrive/My Drive/{path_to_blend}' -E CYCLES -o '/gdrive/My Drive/{output_path}' -s 18 -e 20 -a